# 모듈 선택 및 데이터 로드와 EDA

In [1]:
"""
월간 데이콘 뉴스 토픽 분류 AI 경진대회
- 목적 : 한국어 뉴스 헤드라인을 이용하여, 뉴스의 주제를 분류하는 알고리즘 개발
- 심사 기준 : Accuracy
- 1차 평가(Public Score): 테스트 데이터 중 랜덤 샘플 된 50%로 채점, 대회 기간 중 공개
- 2차 평가(Private Score): 나머지 50 % 테스트 데이터로 채점, 대회 종료 직후 공개
- 최종 순위는 선택된 파일 중에서 채점되므로, 참가자는 제출 창에서 자신이 최종적으로 채점 받고 싶은 파일 2개를 선택해야 함
- 1일 최대 제출 횟수: 3회
- 사용 가능 언어: Python, R
- 모델 학습에서 검증 혹은 평가 데이터셋 활용시(Data Leakage 등) 실격
"""


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
import re
import tqdm as tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, GRU, Bidirectional, LSTM, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

%config InlineBackend.figure_format = 'retina'

# pd.set_option("display.max_seq_items", None)


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")
# label 유형-극성-시제-확실성 이 Target




In [2]:
train.shape, test.shape, sub.shape

((16541, 7), (7090, 2), (7090, 2))

In [3]:
train.head()

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [4]:
train["유형"].value_counts(1), train["극성"].value_counts(1), train["시제"].value_counts(1), train["확실성"].value_counts(1)

(사실형    0.819660
 추론형    0.130041
 대화형    0.034762
 예측형    0.015537
 Name: 유형, dtype: float64,
 긍정    0.954779
 부정    0.034158
 미정    0.011063
 Name: 극성, dtype: float64,
 과거    0.485581
 현재    0.415090
 미래    0.099329
 Name: 시제, dtype: float64,
 확실     0.918445
 불확실    0.081555
 Name: 확실성, dtype: float64)

In [5]:
sub.head()

,ID,label
0,TEST_0000,추론형-긍정-현재-확실
1,TEST_0001,추론형-긍정-현재-확실
2,TEST_0002,추론형-긍정-현재-확실
3,TEST_0003,추론형-긍정-현재-확실
4,TEST_0004,추론형-긍정-현재-확실


In [6]:
"""
유형 컬럼의 포인트
 - 유형은 주로 맨 뒤 단어와 %가 키 포인트라고 판단
 
극성 컬럼의 포인트
 - 유형과 거의 동일함, %자 의미 없어서 제거
 
시제 컬럼의 포인트
 - 유형과 거의 동일함, 과거는 "됐다, 되었다"와 같이
"""

'\n유형 컬럼의 포인트\n - 유형은 주로 맨 뒤 단어와 %가 키 포인트라고 판단\n \n극성 컬럼의 포인트\n - 유형과 거의 동일함, %자 의미 없어서 제거\n \n시제 컬럼의 포인트\n - 유형과 거의 동일함, 과거는 "됐다, 되었다"와 같이\n'

In [7]:
# 1 3 8 14 15 16532 16536 16537

train[train["시제"] == "과거"]["문장"][:]

1        이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...
3        서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...
8            이번 서비스에는 네이버가 자체 개발한 초대규모 AI 하이퍼클로바 기술이 적용됐다.
14       이어 ＂개별 상황에 어떻게 응대해야 할 것인지도 모호한 상황＂이라며 ＂이 같은 부분...
15       사우스포게임즈가 개발한 ＇스컬＇의 경우도 지난해 부산인디커넥트 페스티벌, 글로벌인디...
                               ...                        
16532    국토교통부에 따르면 4월 기준 서울 아파트 미분양 물량은 360가구로 3월(180가...
16536    ＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...
16537    수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...
16538    김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...
16539    1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...
Name: 문장, Length: 8032, dtype: object

In [8]:
train[train["시제"] == "현재"]["문장"][:]

0                    0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.
4                 익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.
5        이같은 변화를 포함해 올해 종부세 과세 대상은 당초 21만4000명에서 12만100...
6        수수꽃다리과로 북한에서 주로 많이 서식한다는 수수꽃다리, 남한의 대표적 라일락으로 ...
7        가장 최근에 있었던, OTT 예능 프로그램 출연으로 일약 스타덤에 올랐던 한 인플루...
                               ...                        
16531    국가의 허리인 중산층이 붕괴하고 있지만, 한국의 중산층 통계는 국민이 체감하는 중산...
16533    계약직으로 근무했던 초단시간 근로자의 일상과 도서관이라는 작은 사회를 통해 우리 사...
16534    뒷좌석에서 동승자나 별도로 안전하게 보관할 수 있는 애견가방 등을 준비하는 게 좋으...
16535    이에 따라 대형 콘서트부터 야외 페스티벌, 실내 공연 등 연이어 오픈 소식이 들려오...
16540                                        《목민심서》의 내용이다.
Name: 문장, Length: 6866, dtype: object

In [9]:
train[train["시제"] == "미래"]["문장"][:]

2        정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...
12       비가 내리는 지역에서는 돌풍과 함께 천둥·번개가 치고 일부 지역에선 우박이 떨어지는...
39                사람 친구는 어느 한쪽에 애인이 생기면 끊어주는 게 쌈빡한 예의 아닐까.
48       계약서 작성이나 분쟁 해결 과정에서 이런 원칙과 기준을 조금이라도 알면 큰 도움이 ...
52                         한빛소프트는 올해도 리폼 이벤트를 계속 진행할 계획이다.
                               ...                        
16504            채권 만기는 내년 1월로, 이를 상환하지 못하면 디폴트 위기를 맞게 된다.
16507     하지만 누군가 나눔과 상생이란 착한 바이러스를 퍼뜨리기 시작한다면 그래도 희망이 있다.
16518       이에 삼양식품이 수출 비중이 높은 만큼 달러 강세의 수혜를 입을 것이라는 전망이다.
16520    불안한 10대들과 공감하며 소통했던 라디오 DJ의 모습부터 사회적 문제 제기와 해결...
16528    5월에는 인천-프랑크푸르트 노선을 5월28일부터 주 4회에서 주 5회로 증편하고, ...
Name: 문장, Length: 1643, dtype: object

# 데이터 전처리 및 학습

In [10]:
label_1 = "유형"
label_2 = "극성"
label_3 = "시제"
label_4 = "확실성"

# "유형" 컬럼 해결

df_1 = train["문장"]

# text 자리에 train["문장"]
# 문장 전처리 함수
def label_1_data(text):
    # 전부 소문자 처리
    text = text.map(lambda x : x.lower())
    # 한글, 영어, 공백, % 제외하고 전부 삭제
    text = text.map(lambda x : re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 a-z%]", "", x))
    # 공백 여러 개 삭제
    text = text.map(lambda x : re.sub("[\s]", " ", x))
    
    return text

df_1 = label_1_data(df_1)


# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ["정부가", "정부는", "정부에서", "이", "같은", "함께", "주", "이어", 
            "앞으로", "이번", "저번", "즉시", "이같은", "년", "월", "일", "및", "더", 
            "마치", "그리고", "그러나", "하지만", "시", "분", "초", "미국은", "미국의", "그래도", 
            "미국", "중국", "중국은", "중국의", "또", "또한", "그", "그래야", "오는", "이외", 
            "서울", "서울은", "서울의", "서울이", "서울에서", "한국", "한국형", "한국은", 
            "한국의", "총", "개", "등", "서울시는", "서울시", "서울시에서", "수출", "수입", "국가의", "국가", "국가는"]
    meaningful_words = [w for w in tokens if not w in stops]
    
    return ' '.join(meaningful_words)

df_1 = df_1.map(lambda x : remove_stopwords(x))

df_1_y = train[label_1]
df_1_y = pd.get_dummies(df_1_y)

# TF-IDF로 적용 / min은 %가 값이 두개 잡히는 경계를 기준
# max는 지니계수의 개념과 같이 생각하면 0.5 정도가 적당하다고 판단
tfvect = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 3), min_df=0.01, max_df=0.5)
df_1_type = tfvect.fit_transform(df_1)
vocab = tfvect.get_feature_names_out()
df_dtm = pd.DataFrame(df_1_type.toarray(), columns=vocab)


X_train, X_test, y_train, y_test = train_test_split(df_dtm, df_1_y, random_state=42, 
                                                   stratify=df_1_y, test_size=0.1)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

model = DecisionTreeClassifier(random_state=42, max_depth=5)
model.fit(X_train, y_train)

df_sub = test["문장"]
df_sub = label_1_data(df_sub)
df_sub = df_sub.map(lambda x : remove_stopwords(x))
df_sub_type = tfvect.transform(df_sub)
df_sub_dtm = pd.DataFrame(df_sub_type.toarray(), columns=vocab)

y_pred = model.predict(df_sub_dtm)
# (y_test == y_pred).mean()

# 대화형 - 사실형 - 예측형 - 추론형

y_predict = pd.DataFrame(y_pred)

# 원핫인코딩 값 되돌리는 함수
def one_hot_back(y_predict):
    y_predict["유형"] = None
    for i in range(len(y_predict["유형"])):
        if y_predict.loc[i, 0] == 1:
            y_predict.loc[i, "유형"] = "대화형"
        if y_predict.loc[i, 1] == 1:
            y_predict.loc[i, "유형"] = "사실형"
        if y_predict.loc[i, 2] == 1:
            y_predict.loc[i, "유형"] = "예측형"
        if y_predict.loc[i, 3] == 1:
            y_predict.loc[i, "유형"] = "추론형"
            
    return y_predict

Answer_1 = one_hot_back(y_predict)
Answer_1 = Answer_1["유형"]
Answer_1
# (y_test == y_pred).mean()

0       사실형
1       사실형
2       사실형
3       사실형
4       사실형
       ... 
7085    사실형
7086    사실형
7087    사실형
7088    사실형
7089    사실형
Name: 유형, Length: 7090, dtype: object

In [11]:
# "극성" 컬럼 해결

df_2 = train["문장"]

# text 자리에 train["문장"]
# 문장 전처리 함수
def label_2_data(text):
    # 전부 소문자 처리
    text = text.map(lambda x : x.lower())
    # 한글, 영어, 공백, % 제외하고 전부 삭제
    text = text.map(lambda x : re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 a-z]", "", x))
    # 공백 여러 개 삭제
    text = text.map(lambda x : re.sub("[\s]", " ", x))
    
    return text

df_2 = label_2_data(df_2)


# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ["정부가", "정부는", "정부에서", "이", "같은", "함께", "주", "이어", 
            "앞으로", "이번", "저번", "즉시", "이같은", "년", "월", "일", "및", "더", 
            "마치", "그리고", "그러나", "하지만", "시", "분", "초", "미국은", "미국의", 
            "미국", "중국", "중국은", "중국의", "또", "또한", "그", "그래야", "오는", "이외", 
            "서울", "서울은", "서울의", "서울이", "서울에서", "한국", "한국형", "한국은", 
            "한국의", "총", "개", "등", "서울시는", "서울시", "서울시에서", "수출", "수입", "국가의", "국가", "국가는"]
    meaningful_words = [w for w in tokens if not w in stops]
    
    return ' '.join(meaningful_words)

df_2 = df_2.map(lambda x : remove_stopwords(x))

df_2_y = train[label_2]
df_2_y = pd.get_dummies(df_2_y)


# TF-IDF로 적용 / min은 %가 값이 두개 잡히는 경계를 기준
# max는 지니계수의 개념과 같이 생각하면 0.5 정도가 적당하다고 판단
tfvect = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 3), min_df=0.01, max_df=0.5)
df_2_type = tfvect.fit_transform(df_2)
vocab = tfvect.get_feature_names_out()
df_dtm = pd.DataFrame(df_2_type.toarray(), columns=vocab)


X_train, X_test, y_train, y_test = train_test_split(df_dtm, df_2_y, random_state=42, 
                                                   stratify=df_2_y, test_size=0.1)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

model = DecisionTreeClassifier(random_state=42, max_depth=5)
model.fit(X_train, y_train)

df_sub = test["문장"]
df_sub = label_2_data(df_sub)
df_sub = df_sub.map(lambda x : remove_stopwords(x))
df_sub_type = tfvect.transform(df_sub)
df_sub_dtm = pd.DataFrame(df_sub_type.toarray(), columns=vocab)

y_pred = model.predict(df_sub_dtm)

# (y_test == y_pred).mean()

#긍정 - 미정 - 부정

y_predict = pd.DataFrame(y_pred)

# 원핫인코딩 값 되돌리는 함수
def one_hot_back(y_predict):
    y_predict["극성"] = None
    for i in range(len(y_predict["극성"])):
        if y_predict.loc[i, 0] == 1:
            y_predict.loc[i, "극성"] = "긍정"
        if y_predict.loc[i, 1] == 1:
            y_predict.loc[i, "극성"] = "미정"
        if y_predict.loc[i, 2] == 1:
            y_predict.loc[i, "극성"] = "부정"
            
    return y_predict

Answer_2 = one_hot_back(y_predict)
Answer_2 = Answer_2["극성"]
Answer_2
# (y_test == y_pred).mean()

0       긍정
1       긍정
2       긍정
3       긍정
4       긍정
        ..
7085    긍정
7086    긍정
7087    긍정
7088    긍정
7089    긍정
Name: 극성, Length: 7090, dtype: object

In [12]:
# "시제" 컬럼 해결

df_3 = train["문장"]

# text 자리에 train["문장"]
# 문장 전처리 함수
def label_3_data(text):
    # 전부 소문자 처리
    text = text.map(lambda x : x.lower())
    # 한글, 영어, 공백, % 제외하고 전부 삭제
    text = text.map(lambda x : re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x))
    # 공백 여러 개 삭제
    text = text.map(lambda x : re.sub("[\s]", " ", x))
    
    return text

df_3 = label_3_data(df_3)


# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ["정부가", "정부는", "정부에서", "이", "같은", "함께", "주", 
            "년", "월", "일", "및", "더", "수출", "수입", "국가의", "국가", "국가는", 
            "마치", "그리고", "그러나", "하지만", "시", "분", "초", "미국은", "미국의", "그래도", 
            "미국", "중국", "중국은", "중국의", "또", "또한", "그", "그래야", "오는", "이외", 
            "서울", "서울은", "서울의", "서울이", "서울에서", "한국", "한국형", "한국은", 
            "한국의", "총", "개", "등", "서울시는", "서울시", "서울시에서"]
    meaningful_words = [w for w in tokens if not w in stops]
    
    return ' '.join(meaningful_words)

df_3 = df_3.map(lambda x : remove_stopwords(x))

df_3_y = train[label_3]
df_3_y = pd.get_dummies(df_3_y)

# TF-IDF로 적용 / min은 %가 값이 두개 잡히는 경계를 기준
# max는 지니계수의 개념과 같이 생각하면 0.5 정도가 적당하다고 판단
tfvect = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 3), min_df=0.01, max_df=0.5)
df_3_type = tfvect.fit_transform(df_3)
vocab = tfvect.get_feature_names_out()
df_dtm = pd.DataFrame(df_3_type.toarray(), columns=vocab)


X_train, X_test, y_train, y_test = train_test_split(df_dtm, df_3_y, random_state=42, 
                                                   stratify=df_3_y, test_size=0.1)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

model = DecisionTreeClassifier(random_state=42, max_depth=5)
model.fit(X_train, y_train)

df_sub = test["문장"]
df_sub = label_3_data(df_sub)
df_sub = df_sub.map(lambda x : remove_stopwords(x))
df_sub_type = tfvect.transform(df_sub)
df_sub_dtm = pd.DataFrame(df_sub_type.toarray(), columns=vocab)

y_pred = model.predict(df_sub_dtm)

# (y_test == y_pred).mean()

# 과거 - 미래 - 현재

y_predict = pd.DataFrame(y_pred)

# 원핫인코딩 값 되돌리는 함수
def one_hot_back(y_predict):
    y_predict["시제"] = None
    for i in range(len(y_predict["시제"])):
        if y_predict.loc[i, 0] == 1:
            y_predict.loc[i, "시제"] = "과거"
        if y_predict.loc[i, 1] == 1:
            y_predict.loc[i, "시제"] = "미래"
        if y_predict.loc[i, 2] == 1:
            y_predict.loc[i, "시제"] = "현재"

    return y_predict

Answer_3 = one_hot_back(y_predict)
Answer_3 = Answer_3["시제"]
Answer_3
# df_3_y
# (y_test == y_pred).mean()

0       현재
1       현재
2       과거
3       현재
4       과거
        ..
7085    현재
7086    현재
7087    현재
7088    현재
7089    현재
Name: 시제, Length: 7090, dtype: object

In [13]:
"""
# 무조건 fit을 해주어서 숫자 연결고리를 맞춰놔야함

max_length = 500
vocab_size = 1000
oov_tok = "<oov>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(df_3)
train_sequence = tokenizer.texts_to_sequences(df_3)
# test_sequence = tokenizer.texts_to_sequences(X_test)
padding_type = "post"
X_train_sp = pad_sequences(train_sequence, maxlen=500, padding=padding_type)
# X_train_sp.shape

embedding_dim = 64
n_class = y_train.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X_train_sp, df_3_y, random_state=42, 
                                                   stratify=df_3_y, test_size=0.1)

# 입력층
model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
                    output_dim=embedding_dim, 
                    input_length=max_length))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Bidirectional(LSTM(units=32)))
model.add(Dense(units=16))
# 출력층
model.add(Dense(units=n_class, activation="softmax"))
# model.summary()

# 컴파일
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics="accuracy")

early_stop = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=100, callbacks=[early_stop])

df_hist = pd.DataFrame(history.history)
df_hist
"""

'\n# 무조건 fit을 해주어서 숫자 연결고리를 맞춰놔야함\n\nmax_length = 500\nvocab_size = 1000\noov_tok = "<oov>"\ntokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_tok)\ntokenizer.fit_on_texts(df_3)\ntrain_sequence = tokenizer.texts_to_sequences(df_3)\n# test_sequence = tokenizer.texts_to_sequences(X_test)\npadding_type = "post"\nX_train_sp = pad_sequences(train_sequence, maxlen=500, padding=padding_type)\n# X_train_sp.shape\n\nembedding_dim = 64\nn_class = y_train.shape[1]\n\nX_train, X_test, y_train, y_test = train_test_split(X_train_sp, df_3_y, random_state=42, \n                                                   stratify=df_3_y, test_size=0.1)\n\n# 입력층\nmodel = Sequential()\nmodel.add(Embedding(input_dim=vocab_size, \n                    output_dim=embedding_dim, \n                    input_length=max_length))\nmodel.add(Bidirectional(LSTM(units=64, return_sequences=True)))\nmodel.add(Bidirectional(LSTM(units=32)))\nmodel.add(Dense(units=16))\n# 출력층\nmodel.add(Dense(units=n_class, activation=

In [14]:
# "확실성" 컬럼 해결

df_4 = train["문장"]

# text 자리에 train["문장"]
# 문장 전처리 함수
def label_4_data(text):
    # 전부 소문자 처리
    text = text.map(lambda x : x.lower())
    # 한글, 영어, 공백, % 제외하고 전부 삭제
    text = text.map(lambda x : re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x))
    # 공백 여러 개 삭제
    text = text.map(lambda x : re.sub("[\s]", " ", x))
    
    return text

df_4 = label_4_data(df_4)


# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ["정부가", "정부는", "정부에서", "이", "같은", "함께", "주", "이어", 
            "앞으로", "이번", "저번", "즉시", "이같은", "년", "월", "일", "및", "더", 
            "마치", "그리고", "그러나", "하지만", "시", "분", "초", "미국은", "미국의", "그래도", 
            "미국", "중국", "중국은", "중국의", "또", "또한", "그", "그래야", "오는", "이외", 
            "서울", "서울은", "서울의", "서울이", "서울에서", "한국", "한국형", "한국은", 
            "한국의", "총", "개", "등", "서울시는", "서울시", "서울시에서", "수출", "수입", "국가의", "국가", "국가는"]
    meaningful_words = [w for w in tokens if not w in stops]
    
    return ' '.join(meaningful_words)

df_4 = df_4.map(lambda x : remove_stopwords(x))

df_4_y = train[label_4]
df_4_y = pd.get_dummies(df_4_y)


# TF-IDF로 적용 / min은 %가 값이 두개 잡히는 경계를 기준
# max는 지니계수의 개념과 같이 생각하면 0.5 정도가 적당하다고 판단
tfvect = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 3), min_df=0.01, max_df=0.5)
df_4_type = tfvect.fit_transform(df_4)
vocab = tfvect.get_feature_names_out()
df_dtm = pd.DataFrame(df_4_type.toarray(), columns=vocab)


X_train, X_test, y_train, y_test = train_test_split(df_dtm, df_4_y, random_state=42, 
                                                   stratify=df_4_y, test_size=0.1)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

model = DecisionTreeClassifier(random_state=42, max_depth=5)
model.fit(X_train, y_train)

df_sub = test["문장"]
df_sub = label_4_data(df_sub)
df_sub = df_sub.map(lambda x : remove_stopwords(x))
df_sub_type = tfvect.transform(df_sub)
df_sub_dtm = pd.DataFrame(df_sub_type.toarray(), columns=vocab)

y_pred = model.predict(df_sub_dtm)

# (y_test == y_pred).mean()

# 불확실 - 확실

y_predict = pd.DataFrame(y_pred)

# 원핫인코딩 값 되돌리는 함수
def one_hot_back(y_predict):
    y_predict["확실성"] = None
    for i in range(len(y_predict["확실성"])):
        if y_predict.loc[i, 0] == 1:
            y_predict.loc[i, "확실성"] = "불확실"
        if y_predict.loc[i, 1] == 1:
            y_predict.loc[i, "확실성"] = "확실"
            
    return y_predict

Answer_4 = one_hot_back(y_predict)
Answer_4 = Answer_4["확실성"]
Answer_4
# (y_test == y_pred).mean()

0       확실
1       확실
2       확실
3       확실
4       확실
        ..
7085    확실
7086    확실
7087    확실
7088    확실
7089    확실
Name: 확실성, Length: 7090, dtype: object

In [17]:
for i in range(len(sub["label"])):
    sub.loc[i, "label"] = f"{Answer_1[i]}-{Answer_2[i]}-{Answer_3[i]}-{Answer_4[i]}"
    
sub.to_csv("submission.csv", index=False)

pd.read_csv("submission.csv")

,ID,label
0,TEST_0000,사실형-긍정-현재-확실
1,TEST_0001,사실형-긍정-현재-확실
2,TEST_0002,사실형-긍정-과거-확실
3,TEST_0003,사실형-긍정-현재-확실
4,TEST_0004,사실형-긍정-과거-확실
...,...,...
7085,TEST_7085,사실형-긍정-현재-확실
7086,TEST_7086,사실형-긍정-현재-확실
7087,TEST_7087,사실형-긍정-현재-확실
7088,TEST_7088,사실형-긍정-현재-확실
